In [1]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import BatchNormalization, Dense, Input, GlobalMaxPooling2D, MaxPooling2D,Flatten,Concatenate
from keras.models import Model

import keras.backend as K
# from data import batch_generator
# from '../Untitled Folder/preprocessing' import batch_generator
import numpy as np

Using TensorFlow backend.


In [2]:
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))

x = vgg16.output
output = GlobalMaxPooling2D()(vgg16.output)
pre_trained = Model(vgg16.input, output)
for layers in vgg16.layers[:5]:
    layers.trainable = False
    
    
    

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
def base_model(input_shape):
    input = Input((input_shape,), name="input")
    
    x = Dense(1024,activation="relu")(input)
    x = Dense(128)(x)
    
    return Model(input,x)

In [4]:
def Final_model(pre_trained):
    inp_shape = K.int_shape(pre_trained.input)[1:]
    conv_feat_size = K.int_shape(pre_trained.output)[-1]
    
    input1 = Input(inp_shape, name="anchor")
    input2 = Input(inp_shape, name="positive")
    input3 = Input(inp_shape, name="negative")
    
    base = base_model(conv_feat_size)
    
    output1 = base(pre_trained(input1))
    output2 = base(pre_trained(input2))
    output3 = base(pre_trained(input3))
    print(output1.shape)
    
    concat = Concatenate(axis=-1)([output1,output2,output3])
    
    return Model([input1,input2,input3],concat)

In [5]:
Final_model = Final_model(pre_trained)

(?, 128)


In [7]:
Final_model.output.shape

TensorShape([Dimension(None), Dimension(384)])

In [6]:
Final_model.summary()
Final_model.output.shape

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
positive (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
negative (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 512)          14714688    anchor[0][0]                     
                                                                 positive[0][0]                   
          

TensorShape([Dimension(None), Dimension(384)])

In [6]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    embedding_size = K.int_shape(y_pred)[-1] // 3
    ind = int(embedding_size * 2)
    a_pred = y_pred[:, :embedding_size]
    p_pred = y_pred[:, embedding_size:ind]
    n_pred = y_pred[:, ind:]
   
    positive_distance = K.sqrt(K.sum(K.square(a_pred - p_pred), axis=-1))
    negative_distance = K.sqrt(K.sum(K.square(a_pred - n_pred), axis=-1))
    
    loss = K.maximum(0.0, positive_distance - negative_distance + alpha)
    return loss
    

In [7]:
Final_model.compile(loss=triplet_loss,optimizer="adam")

In [8]:
path = "../datasets/faceData/"

In [10]:
test_gen = iter(batch_generator(path=path, 
                                target_size=(128,128),
                                horizontal_flip=False,
                                rotate=True,
                                other_transform = False,
                                rescale=1./255,
                                batch_size=10000))

In [ ]:
x,y = next(test_gen)

In [20]:
Final_model.fit(x,y,epochs=5)

Epoch 1/5


TypeError: 'batch_generator' object is not an iterator